In [3]:
import yaml

config_path = '..\\config.yaml'


with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [4]:
import pandas as pd
import json
import os
from tqdm.auto import tqdm
from dotenv import load_dotenv

from openai import OpenAI
from anthropic import Anthropic

load_dotenv(dotenv_path='..\\.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key.")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key.")

openai_clent = OpenAI(api_key = OPENAI_API_KEY)
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

llm_client_map = {
    "gpt-4o-mini": openai_clent,
    "claude-3-sonnet-20240229": anthropic_client
}

def get_llm_client(model_name):
    """
    Get the appropriate LLM client based on the model name.
    Raises an error if the model is not recognized.
    """
    llm_class = llm_client_map.get(model_name)
    if llm_class is None:
        raise ValueError(f"LLM client error: '{model_name}' is not recognized.")
    return llm_class

model_name = config['RAG_model']['model_name']
llm_client = get_llm_client(model_name)

print(f"Successfully initialized {model_name} LLM client.")

Successfully initialized gpt-4o-mini LLM client.


In [5]:
Org =  config['Weaviate']['ORG']
API_KEY = config['Weaviate']['API_KEY']
weaviate_url =  config['Weaviate']['URL']

chunk_size = config['Create_node']['chunk_size']
chunk_overlap = config['Create_node']['chunk_overlap']

documents_file_path = config['Documents_path']

language = config['Language']

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model_name = config.get('Embedding_model', 'sentence-transformers/distiluse-base-multilingual-cased-v2')

if "sentence-transformers" not in embed_model_name:
    print(f"Warning: The model {embed_model_name} is not a sentence-transformer model. Switching to a default.")
    embed_model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v2'

embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model

print(f"Using embedding model: {embed_model_name}")

c:\Users\rkizm\.virtualenvs\legal-advisor-rag-a8TsuGdb\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using embedding model: sentence-transformers/distiluse-base-multilingual-cased-v2


In [5]:
# tx = 'how are you?'
# embed_model.get_text_embedding(tx) == embed_model.get_query_embedding(tx)

# emb1 = embed_model.get_text_embedding(tx)
# emb2 = embed_model.get_query_embedding(tx)

# embed_model.similarity(embedding1=emb1, embedding2=emb2)

# Ingestion

In [6]:
# import weaviate

# def get_weaviate_client(api_key, url):
#     return weaviate.Client(
#             url=weaviate_url,
#             auth_client_secret=weaviate.AuthApiKey(api_key=api_key)
#         )

# # client
# client = get_weaviate_client(API_KEY, weaviate_url)

# print(f"Client is ready: {client.is_ready()}")

# # DELETING all info from DB
# client.schema.delete_class(Org)

c:\Users\rkizm\.virtualenvs\legal-advisor-rag-a8TsuGdb\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\rkizm\AppData\Local\Temp\ipykernel_17056\3416258396.py:4: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  return weaviate.

Client is ready: True


In [7]:
# import weaviate
# from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex, Document
# from llama_index.vector_stores.weaviate import WeaviateVectorStore
# from transformers import BertTokenizer
# import re

# def load_documents(file_path):
#     return SimpleDirectoryReader(file_path).load_data()

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


# def count_tokens(text):
#     """Counting the number of tokens in a text"""
#     tokens = tokenizer.tokenize(text)
#     return len(tokens)


# def split_large_text(text, max_tokens=4000):
#     """Split the text into fragments no larger than max_tokens"""
#     words = text.split()
#     split_texts = []
#     current_chunk = []

#     for word in words:
#         current_chunk.append(word)
#         if count_tokens(' '.join(current_chunk)) >= max_tokens:
#             split_texts.append(' '.join(current_chunk))
#             current_chunk = []

#     if current_chunk:
#         split_texts.append(' '.join(current_chunk))

#     return split_texts


# def split_text_by_paragraphs(text, max_tokens=4000):
#     """Splitting text into paragraphs of no more than max_tokens"""
#     paragraphs = text.split('\n\n')
#     new_nodes = []
#     current_text = ""

#     for para in paragraphs:
#         if count_tokens(current_text + para) < max_tokens:
#             current_text += para + "\n\n"
#         else:
#             if current_text:
#                 new_nodes.append(current_text.strip())
#             current_text = para + "\n\n"

#     if current_text.strip():
#         new_nodes.append(current_text.strip())

#     final_nodes = []
#     for node in new_nodes:
#         if count_tokens(node) > max_tokens:
#             split_nodes = split_large_text(node, max_tokens)
#             final_nodes.extend(split_nodes)
#         else:
#             final_nodes.append(node)

#     return final_nodes


# def create_nodes(documents, max_tokens=4000):
#     all_nodes = []
#     for doc in documents:
#         law_name = doc.metadata['file_name'].replace('.txt', '')
#         law_text = doc.text
#         nodes_list = re.split(r'\n(?=Статья \d+\.)', law_text.strip())
        
#         for node_text in nodes_list:
#             if count_tokens(node_text) <= max_tokens:
#                 all_nodes.append(Document(text=node_text, metadata={'file_name': law_name}))
#             else:
#                 split_nodes = split_text_by_paragraphs(node_text, max_tokens=4000)
#                 for split_node in split_nodes:
#                     all_nodes.append(Document(text=split_node, metadata={'file_name': law_name}))

#     return all_nodes


# def connect_index(weaviate_client):
#     vector_store = WeaviateVectorStore(
#         weaviate_client=weaviate_client,
#         index_name=Org
#     )

#     storage_context = StorageContext.from_defaults(vector_store=vector_store)
#     index = VectorStoreIndex([], storage_context=storage_context)
#     return index


# def insert_nodes_index(index, nodes):
#     index.insert_nodes(nodes)


# client = weaviate.connect_to_wcs(
#             cluster_url=weaviate_url,
#             auth_credentials=weaviate.AuthApiKey(api_key=API_KEY)
# )
# print("Weaviate client is ready:", client.is_ready())

# documents = load_documents(documents_file_path)
# nodes = create_nodes(documents)

# index = connect_index(weaviate_client=client)
# insert_nodes_index(index, nodes=nodes)

# client.close()
# print("Weaviate client connection closed.")

c:\Users\rkizm\.virtualenvs\legal-advisor-rag-a8TsuGdb\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Weaviate client is ready: True
Weaviate client connection closed.


# RAG flow

In [7]:
import weaviate
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore

def connect_index(weaviate_client):
    vector_store = WeaviateVectorStore(
        weaviate_client=weaviate_client,
        index_name=Org
    )

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex([], storage_context=storage_context)
    return index

weaviate_client = weaviate.connect_to_wcs(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.AuthApiKey(api_key=API_KEY)
    )
print("Client is ready:", weaviate_client.is_ready())

index = connect_index(weaviate_client=weaviate_client)

c:\Users\rkizm\.virtualenvs\legal-advisor-rag-a8TsuGdb\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


Client is ready: True


In [8]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index,
    vector_store_query_mode="hybrid",
    similarity_top_k = config['Retriever']['similarity_top_k'],
    alpha = 0.9, #config['Retriever']['alpha'],
    similarity_threshold = config['Retriever']['similarity_threshold'],
    )

def search_nodes(query):
    return retriever.retrieve(query)


def build_prompt(query, search_results):

    nodes_with_scores = [(node, node.score) for node in search_results]
    sorted_nodes = sorted(nodes_with_scores, key=lambda x: x[1], reverse=True)
    sorted_nodes_only = [node for node, score in sorted_nodes]

    context = ""
    
    for nod in sorted_nodes_only:
        context = context + nod.metadata['file_name'].replace('txt', '') + '\n' + nod.text + "\n\n"

    prompt_template = config['RAG_model']['prompt_template']
    prompt = prompt_template.format(language = language, question=query, context=context).strip()
    return prompt

def llm_response(prompt):
    response = llm_client.chat.completions.create(
        model=config['RAG_model']['model_name'],
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search_nodes(query)
    prompt = build_prompt(query, search_results)
    answer = llm_response(prompt)
    return answer

In [66]:
query = "В каких случаях предусмотрена уголовная ответственность за подделку или уничтожение идентификационного номера транспортного средства, и какие виды наказания могут быть применены за совершение данного преступления?"
for res in search_nodes(query)[:2]:
    print('***', res.text)

*** Статья 380. Подделка, уничтожение идентификационного номера транспортного средства
 
Подделка или уничтожение идентификационного номера, номера кузова, шасси, транспортного средства – 
наказываются штрафом от 500 до 1000 расчетных показателей либо лишением свободы на срок от двух до пяти лет.
 
*** Статья 578. Ответственность за вред, причиненный транспортному средству
 
В случае гибели или повреждения арендованного транспортного средства арендатор обязан возместить арендодателю причиненные убытки, если последний докажет, что гибель или повреждение транспортного средства произошли по обстоятельствам, за которые арендатор отвечает в соответствии с законом или договором аренды.
 


In [19]:
answer = rag(query)
print(answer)

Criminal liability for the forgery or destruction of a vehicle's identification number is established in Article 380 of the Criminal Code of the Kyrgyz Republic. The punishment for this crime can include a fine ranging from 500 to 1000 calculated indicators or imprisonment for a term of two to five years.


# Evaluation-data-generation

In [66]:
# from random import randint

# numder_of_articles = 50

# f = True
# while f:
#     l = [randint(1, len(nodes)) for _ in range(numder_of_articles)]
#     if len(set(l)) == numder_of_articles:
#         f=False

# sample = [nodes[i] for i in l]

# data = []

# for node in sample:
#     data.append({
#         'id': node.id_,
#         'law_name': node.metadata['file_name'],
#         'article': node.text
#     })

# df = pd.DataFrame(data)
# df.to_csv('..\\data\\ground_truth.csv', index=False)
# df.shape

(50, 3)

In [67]:
# df = pd.read_csv('..\\data\\ground_truth.csv')
# documents = df.to_dict(orient='records')

In [83]:
# question_generator_prompt_template = """
# You emulate a user of our legal advisor on legislation of Kyrgyz Republic application.
# Formulate 3 questions this user might ask based on a provided article of the law.
# Make the questions specific to the provided record. Don't reffer to the number of the article of the law.
# The record should contain the answer to the questions, and the questions should be complete and not too short.
# Use as fewer words as possible from the record. 

# The record:

# Law name: {law_name}
# Article of the law: {article}

# Provide the output in parsable JSON without using code blocks:

# {{"questions": ["question1", "question2", "question3"]}}
# """.strip()

In [90]:
# def llm_response(prompt):
#     response = llm_client.chat.completions.create(
#         model=config['RAG_model']['model_name'],
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content


# def generate_questions(doc):
#     question_generator_prompt = question_generator_prompt_template.format(**doc)
#     json_response = llm_response(question_generator_prompt)
#     return json_response


# results = {}

# for doc in tqdm(documents): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue

#     questions_raw = generate_questions(doc)
#     questions = json.loads(questions_raw)
#     results[doc_id] = questions['questions']

  0%|          | 0/50 [00:00<?, ?it/s]

In [91]:
# final_results = []

# for doc_id, questions in results.items():
#     for q in questions:
#         final_results.append((doc_id, q))

# df_questions = pd.DataFrame(final_results, columns=['id', 'question'])
# df_questions.to_csv('../data/ground-truth-questions.csv', index=False)
# df_questions.shape

(150, 2)

# Retrieval evaluation

In [12]:
# # Getting all nodes from weaviate

# import weaviate

# # Initialize the Weaviate client
# client = weaviate.Client(
#     url=weaviate_url,
#     auth_client_secret=weaviate.AuthApiKey(api_key=API_KEY)
# )

# # Function to get all classes in the Weaviate instance
# def get_all_classes(client):
#     schema = client.schema.get()
#     classes = [cls['class'] for cls in schema['classes']]
#     return classes

# # Function to get all properties of a specific class
# def get_class_properties(client, class_name):
#     schema = client.schema.get()
#     for cls in schema['classes']:
#         if cls['class'] == class_name:
#             return [prop['name'] for prop in cls['properties']]
#     return []

# # Function to get all text nodes from a specific class with pagination
# def get_all_text_nodes(client, class_name, properties):
#     properties_str = ' '.join(properties)
#     all_nodes = []
#     limit = 100
#     offset = 0

#     while True:
#         query = """
#         {
#             Get {
#                 %s(
#                     limit: %d
#                     offset: %d
#                 ) {
#                     %s
#                     _additional {
#                         id
#                     }
#                 }
#             }
#         }
#         """ % (class_name, limit, offset, properties_str)

#         response = client.query.raw(query)
#         nodes = response['data']['Get'][class_name]

#         if not nodes:
#             break

#         all_nodes.extend(nodes)
#         offset += limit

#     return all_nodes

# # Get all classes
# classes = get_all_classes(client)
# print("Available classes:", classes)

# # Choose a class to query (for example, the first class)
# if classes:
#     class_name = Org #classes[0]
#     properties = get_class_properties(client, class_name)
#     print(f"Properties of class '{class_name}':", properties)

#     # Get and print all text nodes from the chosen class
#     if properties:
#         text_nodes = get_all_text_nodes(client, class_name, properties)
#         print("Number of text nodes retrieved:", len(text_nodes))
#         # Print first few nodes as an example
#         print("First few text nodes:", text_nodes[:5])
#     else:
#         print(f"No properties found for class '{class_name}'.")
# else:
#     print("No classes found in the Weaviate schema.")


# # merging questions with articles on 'id'
# df_ar = pd.read_csv('..\\data\\ground_truth.csv')
# df_questions = pd.read_csv('../data/ground-truth-questions.csv')
# df = pd.merge(df_questions, df_ar, on='id', how='left')


# # merging 'new_id' from text_nodes with questions
# df_weaviate = pd.DataFrame(text_nodes)
# df_weaviate._additional = df_weaviate._additional.apply(lambda i: i['id'])
# df_weaviate=df_weaviate[['_additional', 'text']]
# df_weaviate.columns = ['new_id', 'article']
# df = pd.merge(df, df_weaviate, on='article', how='left')


# df.to_csv('..\\data\\ground_truth_data.csv', index=False)
# df.sample(2)

,id,question,law_name,article,new_id
9,74688912-792b-4de9-966a-ee2c76d91006,What can a lender demand if the borrower fails...,Гражданский Кодекс КР Часть II,Статья 730. Обеспечение исполнения обязательст...,3fdb1a03-dc6d-4c48-9d66-aa632b173948
122,21ec195b-1ef9-4bfb-b4be-c2d8abf2e165,Is there a minimum period for filing an appell...,Гражданский процессуальный кодекс Кыргызской Р...,Статья 237. Обжалование заочного решения\r\n1....,d57aae71-032b-4a61-a513-e12f717ecbed


In [14]:
df = pd.read_csv('..\\data\\ground_truth_data.csv')
ground_truth = df.to_dict(orient='records')
ground_truth[0].keys()

dict_keys(['id', 'question', 'law_name', 'article', 'new_id'])

In [2]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\rkizm\.virtualenvs\legal-advisor-rag-a8TsuGdb\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# %pip install git+https://github.com/alekssamos/yandexfreetranslate.git
# %pip install yandexfreetranslate

In [ ]:
from yandexfreetranslate import YandexFreeTranslate
yt = YandexFreeTranslate(api='ios') #Работает только так

def ru(txt):
  return yt.translate("en", "ru", txt)

def en(txt):
  return yt.translate("ru", "en", txt)

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['new_id']
        results = search_function(q)
        relevance = [d.id_ == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [21]:
alphas = [0.1 * i for i in range(1, 11)]

df_retrieval_eval = pd.DataFrame(columns=['alpha', 'hit_rate', 'mrr'])
for alpha in alphas:
    retriever = VectorIndexRetriever(
        index,
        vector_store_query_mode="hybrid",
        similarity_top_k = config['Retriever']['similarity_top_k'],
        alpha = round(alpha, 1),
        similarity_threshold = 5,
        )
    
    def search_nodes(query):
        return retriever.retrieve(query['question'])

    eval_res = evaluate(ground_truth, search_nodes)

    df_retrieval_eval.loc[len(df_retrieval_eval)] = {'alpha': round(alpha, 1),
                                                    'hit_rate': eval_res['hit_rate'],
                                                    'mrr':eval_res['mrr']}


df_retrieval_eval_ru = pd.DataFrame(columns=['alpha', 'hit_rate_ru_query', 'mrr_ru_query'])
for alpha in alphas:
    retriever = VectorIndexRetriever(
        index,
        vector_store_query_mode="hybrid",
        similarity_top_k = config['Retriever']['similarity_top_k'],
        alpha = round(alpha, 1),
        similarity_threshold = 5,
        )
    
    def search_nodes(query):
        return retriever.retrieve(ru(query['question']))

    eval_res = evaluate(ground_truth, search_nodes)

    df_retrieval_eval_ru.loc[len(df_retrieval_eval_ru)] = {'alpha': round(alpha, 1),
                                                           'hit_rate_ru_query': eval_res['hit_rate'],
                                                           'mrr_ru_query':eval_res['mrr']}
df_retrieval_eval = pd.merge(df_retrieval_eval, df_retrieval_eval_ru, on='alpha', how='left')


df_retrieval_eval_rerank = pd.DataFrame(columns=['alpha', 'hit_rate_rerank', 'mrr_rerank'])
for alpha in alphas:
    retriever = VectorIndexRetriever(
        index,
        vector_store_query_mode="hybrid",
        similarity_top_k = config['Retriever']['similarity_top_k'],
        alpha = round(alpha, 1),
        similarity_threshold = 10,
        node_postprocessors=[colbert_reranker]
        )
    
    def search_nodes(query):
        return retriever.retrieve(ru(query['question']))

    eval_res = evaluate(ground_truth, search_nodes)

    df_retrieval_eval_rerank.loc[len(df_retrieval_eval_rerank)] = {'alpha': round(alpha, 1),
                                                                   'hit_rate_rerank': eval_res['hit_rate'],
                                                                   'mrr_rerank':eval_res['mrr']}
df_retrieval_eval = pd.merge(df_retrieval_eval, df_retrieval_eval_rerank, on='alpha', how='left') 

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

In [22]:
df_retrieval_eval

,alpha,hit_rate,mrr,hit_rate_ru_query,mrr_ru_query,hit_rate_rerank,mrr_rerank
0,0.1,0.626667,0.382622,0.686667,0.484405,0.686667,0.484405
1,0.2,0.626667,0.382622,0.720000,0.516323,0.720000,0.516323
2,0.3,0.626667,0.382622,0.753333,0.546304,0.753333,0.546304
3,0.4,0.626667,0.382622,0.793333,0.563156,0.793333,0.563156
4,0.5,0.633333,0.389288,0.806667,0.564524,0.806667,0.564524
5,0.6,0.633333,0.389288,0.800000,0.548487,0.800000,0.548487
6,0.7,0.640000,0.389955,0.766667,0.513161,0.766667,0.513161
7,0.8,0.640000,0.390122,0.726667,0.487958,0.726667,0.487958
8,0.9,0.640000,0.390622,0.666667,0.438357,0.666667,0.438357
9,1.0,0.640000,0.390955,0.633333,0.389579,0.633333,0.389579


Translations of questions improves the rates of retriever. I will use alpha = 0.5